In [18]:
#imports
import numpy as np
import os
import pandas as pd
import json
# import scikit-learn as sklearn

In [19]:
# load the datasets

def loop_slices(path, num_slices=20):
  cnt = 0
  mpd_playlists = []
  filenames = os.listdir(path)
  for fname in sorted(filenames):
    print(fname)
    if fname.startswith("mpd.slice.") and fname.endswith(".json"):
        cnt += 1
        fullpath = os.sep.join((path, fname))
        f = open(fullpath)
        js = f.read()
        f.close()
        current_slice = json.loads(js)      
        
        # Create a list of all playlists
        for playlist in current_slice['playlists']:
            mpd_playlists.append(playlist)      
    
        if cnt == num_slices:
            break  
    return mpd_playlists
# Path where the json files are extracted
million_playlists_json = 'data/spotify_million_playlist_dataset/data/'
playlists = loop_slices(million_playlists_json, num_slices=20)
#convert to a df
df = pd.json_normalize(playlists)

# idea for how to combine features https://medium.com/analytics-vidhya/music-recommender-system-part-2-ff4c3f54cba3

#load our spotify features dataset
spotify_features = pd.read_csv("data/SpotifyFeatures.csv")

mpd.slice.0-999.json


In [23]:
# display(spotify_features)
# playlists

In [24]:
display(df)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,old,false,995,1507852800,41,40,1,"[{'pos': 0, 'artist_name': 'Katrina', 'track_u...",8,9917901,36,NaN
996,Daze,false,996,1479254400,17,17,1,"[{'pos': 0, 'artist_name': 'PARTYNEXTDOOR', 't...",13,3699248,15,NaN
997,rap,false,997,1410307200,119,98,1,"[{'pos': 0, 'artist_name': 'LoveRance', 'track...",63,27538723,82,NaN
998,Country,false,998,1507939200,108,75,1,"[{'pos': 0, 'artist_name': 'Hunter Hayes', 'tr...",37,24950143,40,NaN
